# 코더블 첫번째 숙제
### Perceptron Training

저희는 미팅에서 설명드렸던 것과 같이 Perceptron 이라는 Linear Classifier을 implement하여 Binary Classification 작업을 할 것입니다.


## Import Module

In [1]:
import numpy as np
import pickle
import time

## Dependency functions
DO NOT EDIT.

### Data preprocessing

In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def load_dataset(filename, extra=False):
    A = unpickle(filename)
    X = A[b'data'] / 255 # feature scale the data to range 0-1
    Y = A[b'labels']

    # make small dataset for knn
    if extra:
        X = X[: 1000]
        Y = Y[: 1000]

    test_size = int(0.25 * len(X)) # set aside 25% for testing
    X_test = X[:test_size]
    Y_test = Y[:test_size]
    X = X[test_size:]
    Y = Y[test_size:]

    animals = [2,3,4,5,6,7]
    Y = np.array([Y[i] in animals for i in range(len(Y))])
    Y_test = np.array([Y_test[i] in animals for i in range(len(Y_test))])

    return X,Y,X_test,Y_test

### Accuracy function

In [3]:
def compute_accuracies(predicted_labels,dev_labels):
    if not isinstance(predicted_labels, list):
        print('Predict labels must be list')
        assert False
    yhats = predicted_labels
    if len(yhats) != len(dev_labels):
        print('Predict labels must have the same length as actual labels!')
        assert False
    if not all([(y == 0 or y == 1) for y in yhats]):
        print('Predicted labels must only contain 0s or 1s')
        assert False
    accuracy = np.mean(yhats == dev_labels)
    tp = np.sum([yhats[i] == dev_labels[i] and yhats[i] == 1 for i in range(len(yhats))])
    precision = tp / np.sum([yhats[i]==1 for i in range(len(yhats))])
    recall = tp / (np.sum([yhats[i] != dev_labels[i] and yhats[i] == 0 for i in range(len(yhats))]) + tp)
    f1 = 2 * (precision * recall) / (precision + recall)

    print("Accuracy:",accuracy)
    print("F1-Score:",f1)
    print("Precision:",precision)
    print("Recall:",recall)

    return accuracy,f1,precision,recall


## Your functions!

### Train Perceptron Algorithm

#### hints
1. In this case of perceptron training algorithm, we will be using np.sign() function as a scoring function.
    * If sign(Wx + b) > 0, prediction = 1
    * If sign(Wx + b) <= 0, prediction = 0
2. Initialize the weights and bias a zeros.
3. You can either use weight and bias as seperate variables or use the "bias trick"!
4. weight update function: weights(t + 1) = weights(t) + learning_rate * (expected_i – predicted_) * input_i


In [81]:
"""
train_set = A Numpy array of 32x32x3 images of shape [7500, 3072].
            this can be thought of as a list of N 1 x D vectors where D stands
            for the dimension of the image where 32*32*3 = 3072 and the 
            N is the total number of images in the training set = 7500.

train_labels = List of labels coresponding to the training set.
               Range of labels [1, 0] if the image contains a picture of an animal, label = 1, else = 0.

dev_set = A Numpy array of 32*32*3 images of shape[2500, 3072].
          It is the same format as train_set.
"""

# def trainPerceptron(train_set, train_labels, learning_rate, max_iter):
#     # TODO: Write your code here
#     # return the trained weight and bias parameters
    
#     K = 2
#     D = len(train_set[0]) #3072
#     N = len(train_set)    #7500
#     W = np.zeros((K, D))  #2, 3072
#     b = np.zeros((K, 1))

    #K X D, D X 1, , K + 1
    
#     for _ in range(max_iter): 
#         for i in range(len(train_set)):
# #             prediction = 1 if np.sign(np.dot(W, train_set[i])+b).all() > 0 else 0
            
#             score = np.sign(np.dot(W, train_set[i])+b).any()
            
#             if train_labels[i] == 1: prediction = 1
#             else: prediction = -1
    
#             if prediction != score :
#                 W += learning_rate * (train_labels[i] - prediction) * train_set[i]
#                 b += learning_rate * prediction
        
#     return W, b

def trainPerceptron(train_set, train_labels, learning_rate, max_iter):
    # TODO: Write your code here
    # return the trained weight and bias parameters
    W=[0]*(len(train_set[0]))
    b=0
    for m in range(max_iter):
        for i in range(len(train_set)):
            img = train_set[i]
            sign = np.sign(np.dot(W,img)+b)
        
            if train_labels[i]==1:
                sign_ = 1
            else:
                sign_ = -1

            if sign != sign_:
                W += learning_rate*sign_*img
                b += learning_rate*sign_
    return W, b

### Train and Predict w/ Perceptron model

In [94]:
def classifyPerceptron(train_set, train_labels, dev_set, learning_rate, max_iter):
    # TODO: Write your code here
    # Train perceptron model and return predicted labels of development set
    
    W, b = trainPerceptron(train_set, train_labels, learning_rate, max_iter)
    
    predictions = []
#     for _ in range(max_iter):
    for i in range(len(dev_set)):
        prediction = 1 if np.sign((np.dot(W, dev_set[i]) + b)).any() > 0 else 0 
#         predictions.append(prediction)
        
        if prediction == 1: predictions.append(1)
        else: predictions.append(0)
        
    return predictions
        
# def classifyPerceptron(train_set, train_labels, dev_set, learning_rate, max_iter):
#     # TODO: Write your code here
#     # Train perceptron model and return predicted labels of development set
#     W,b = trainPerceptron(train_set, train_labels, learning_rate, max_iter)
#     dev_labels = []

#     for i in range(len(dev_set)):
#         img = dev_set[i]
#         if np.dot(W,img)+b > 0:
#             sign = 1
#         else:
#             sign = -1
#         if sign == 1:
#             dev_labels.append(1)
#         else:
#             dev_labels.append(0)
#     return dev_labels


## Hyperparameters

Change as you wish to improve the performance!

In [95]:
LR = 0.01
EPOCHS = 10

## Train!

#### Try to get over 0.75 accuracy 

In [96]:
def train(lrate, epochs):
    start_time = time.perf_counter()

    train_set, train_labels, dev_set,dev_labels = load_dataset("mp5_data")
    pred_p = classifyPerceptron(train_set, train_labels, dev_set, lrate, epochs)
    accuracy,f1,precision,recall = compute_accuracies(pred_p, dev_labels)

    end_time = time.perf_counter()

    print(f'Time taken for Perceptron to execute training and classification: {end_time - start_time:.3f} seconds')


In [97]:
train(LR, EPOCHS)

Accuracy: 0.5964
F1-Score: 0.7471811576046105
Precision: 0.5964
Recall: 1.0
Time taken for Perceptron to execute training and classification: 0.434 seconds


In [52]:
train_set, train_labels, dev_set,dev_labels = load_dataset("mp5_data")

In [148]:
len(train_set[0])

3072

In [141]:
K = 2
D = len(train_set[0]) #3072
N = len(train_set)    #7500
W = np.zeros((K, D))
b = np.zeros((K, 1))
len(W[1])

3072